#

In [1]:
import pandas as pd

In [2]:
big = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
small = pd.DataFrame({'department':['A','B'], 'total':[3,4]})

In [3]:
display("big",big)
display("small",small)

In [12]:
pd.merge(big,small) 
big.merge(small)
pd.merge(small,big) 
small.merge(big)

In [16]:
big.merge(small,on='department')

`# 예제2`

In [30]:
big = pd.DataFrame({
    'department':['A']*4+['B']*4, 
    'gender':(['male']*2+['female']*2)*2,
    'repeat':['Yes','No']*4,
    'count':[1,2,3,1,4,5,2,0]}
)
small = pd.DataFrame({'department':['A','B'], 'total':[3,4]})

In [31]:
big

In [25]:
s1 = pd.DataFrame({
    'name':['이성계','이방원','정몽주','하륜'],
    'desc':['왕','이성계의 아들','이성계의 신하','이방원의 부하']})
s1

In [26]:
s2 = pd.DataFrame({
    'name':['이방원','이성계','하륜'],
    'desc':['왕','왕의 아버지','이성계의 신하']})
s2

In [28]:
s1.merge(s2,on='name')

In [16]:
학생 = pd.DataFrame({
    '이름': ['지민', '태형', '윤기', '호석'],
    '동아리': ['음악', '미술', '체육', '음악']
})

동아리 = pd.DataFrame({
    '동아리': ['음악', '미술', '체육', '과학'],
    '선생님': ['김선생님', '이선생님', '박선생님', '최선생님']
})

결과 = 학생.merge(동아리, left_on='동아리', right_on='동아리',how='outer')
결과

In [10]:
학생 = pd.DataFrame({
    '이름': ['지민', '태형', '윤기', '호석'],
    '동아리': ['음악', '미술', '체육', '음악'],
    '학년': [1, 2, 3, 1]
})

동아리 = pd.DataFrame({
    '동아리': ['음악', '미술', '체육', '과학'],
    '학년': [1, 2, 3, 4],
    '선생님': ['김선생님', '이선생님', '박선생님', '최선생님']
})

결과 = 학생.merge(동아리)
결과